In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from nltk import word_tokenize, bigrams, trigrams, FreqDist
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.corpus import stopwords
import string

# Baixar recursos necessários
nltk.download('punkt')
nltk.download('stopwords')

# Função para pré-processamento
def preprocess_text(text):
    stop_words = set(stopwords.words('portuguese'))  # Ajuste conforme a língua do dataset
    text = text.lower()  # Convertendo para minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removendo pontuação
    tokens = word_tokenize(text)  # Tokenização
    tokens = [word for word in tokens if word not in stop_words and word.isalpha()]  # Removendo stopwords
    return tokens



In [ ]:
# Carregar um dataset de texto (exemplo com CSV)
df = pd.read_csv("dataset_texto.csv")  # Substituir pelo caminho correto do dataset

# Considerar apenas uma coluna de texto (ajustar conforme necessário)
df.dropna(inplace=True)
text_data = ' '.join(df['coluna_texto'].tolist())  # Unindo textos

# Aplicar pré-processamento
tokens = preprocess_text(text_data)

# Criando bigramas e trigramas
bigrams_list = list(bigrams(tokens))
trigrams_list = list(trigrams(tokens))

# Criando modelo de N-grama com MLE
n = 3  # Define o tamanho do modelo (trigramas, por exemplo)
train_data, padded_vocab = padded_everygram_pipeline(n, [tokens])
model = MLE(n)
model.fit(train_data, padded_vocab)

# Exibir algumas probabilidades de bigramas
def display_ngram_probabilities(model, context, top_n=5):
    print(f"Contexto: {context}")
    candidates = {word: model.score(word, context) for word in model.vocab}
    sorted_candidates = sorted(candidates.items(), key=lambda x: x[1], reverse=True)[:top_n]
    for word, prob in sorted_candidates:
        print(f"{context} -> {word}: {prob:.4f}")



In [ ]:
# Exemplo de contexto
display_ngram_probabilities(model, ('exemplo',), top_n=5)

# Visualizando distribuição de frequência de bigramas
fdist = FreqDist(bigrams_list)
fdist.plot(20, title="Top 20 Bigramas Frequentes")

# Visualizando distribuição de frequência de trigramas
fdist_tri = FreqDist(trigrams_list)
fdist_tri.plot(20, title="Top 20 Trigramas Frequentes")
